In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project") this is not working!
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print(ws)

Workspace name: quick-starts-ws-126661
Azure region: southcentralus
Subscription id: 0c5a644d-c5ce-4e3b-bf42-4cb265317817
Resource group: aml-quickstarts-126661
Workspace.create(name='quick-starts-ws-126661', subscription_id='0c5a644d-c5ce-4e3b-bf42-4cb265317817', resource_group='aml-quickstarts-126661')


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-17T20:02:23.231000+00:00', 'errors': None, 'creationTime': '2020-11-17T20:02:20.008518+00:00', 'modifiedTime': '2020-11-17T20:02:36.089504+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.01,0.1,1,10,100,500,1000),
        '--max_iter': choice(50,100,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",compute_target=compute_target,vm_size='STANDARD_D2_V2',entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
#                                      max_concurrent_runs=4,
                                     policy=policy,
                                     estimator=est,
                                     max_total_runs=16
                                    )

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())
# model = best_run.register_model(model_name='sklearn_log_reg_hyperdrive', model_path='outputs/model.joblib')


best run details : {'runId': 'HD_e5f45365-b5c6-4bc9-a46e-f0e442810526_1', 'target': 'cpu-cluster', 'status': 'Completed', 'startTimeUtc': '2020-11-16T08:27:55.010546Z', 'endTimeUtc': '2020-11-16T08:30:08.157985Z', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '64c66cd8-3994-4627-8a14-e0b6289c3ef7', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'useAbsolutePath': False, 'arguments': ['--C', '0.1', '--max_iter', '100'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'cpu-cluster', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'priority': None, 'environment': {'name': 'Experiment udacity-project Environment', 'version': 'Autosave_2020-11-16T08:24:23Z_98194a8d', 'python': {'interpreterPath': 'python

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [7]:
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x,y = clean_data(ds)
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)
import pandas as pd
xy_train = pd.concat([x_train,y_train],axis=1)

xy_train.to_csv('training/xy_train.csv')
ds1 = ws.get_default_datastore()
ds1.upload(src_dir='training',target_path='training')
xy_train_ds = Dataset.Tabular.from_delimited_files(ds1.path('training/xy_train.csv'))

Uploading an estimated of 1 files
Uploading training/xy_train.csv
Uploaded training/xy_train.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target = compute_target,
    training_data=xy_train_ds,
    label_column_name='y',
    n_cross_validations = 3,
    primary_metric = 'accuracy',
    enable_early_stopping = False,
)

In [9]:
# Submit your automl run

remote_run = exp.submit(config= automl_config, show_output= True)

Running on remote.
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_a289b456-9013-4269-bcca-000b6b2fa00a

Received interrupt. Returning now.

KeyboardInterrupt: 

In [17]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: udacity-project,
Id: AutoML_d2e9b932-a6a1-4942-8307-7f63772e78b9_29,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_leaf=0.035789473684210524,
                                                                                                  min_samples_split=0.01,
    